## 幾種思路

思路1：TF-IDF + 機器學習分類器
直接使用TF-IDF對文本提取特徵，並使用分類器進行分類。在分類器的選擇上，可以使用SVM、LR、或者XGBoost。

思路2：FastText
FastText是入門款的詞向量，利用Facebook提供的FastText工具，可以快速構建出分類器。

思路3：WordVec + 深度學習分類器
WordVec是進階款的詞向量，並通過構建深度學習分類完成分類。深度學習分類的網絡結構可以選擇TextCNN、TextRNN或者BiLSTM。

思路4：Bert詞向量
Bert是高配款的詞向量，具有強大的建模學習能力。

## 獲取6種模型的資料集

In [175]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import re
path = '../data/0414/review_data(seg+pos+stopwords)_a+n+v.csv'
df = pd.read_csv(path)

### 檢查重複值、空值

In [176]:
#印出重複資料
print(df[df.duplicated()])

Empty DataFrame
Columns: [reviews, value, comfort, location, cleanliness, service, facilities, ws_pos_reviews, filtered, filtered_word]
Index: []


In [177]:
#移除重複值
#df = df.drop_duplicates()
#print(df.shape)

In [178]:
#印出空值資料
#df_train[df_train.isnull().T.any()]

### 切分為6個資料集

In [179]:
def split_df(df):
    df_value = df[['value','filtered_word']]
    df_value.rename(columns={'value': 'label'}, inplace=True)
    df_comfort = df[['comfort','filtered_word']]
    df_comfort.rename(columns={'comfort': 'label'}, inplace=True)
    df_location = df[['location','filtered_word']]
    df_location.rename(columns={'location': 'label'}, inplace=True)
    df_cleanliness = df[['cleanliness','filtered_word']]
    df_cleanliness.rename(columns={'cleanliness': 'label'}, inplace=True)
    df_service = df[['service','filtered_word']]
    df_service.rename(columns={'service': 'label'}, inplace=True)
    df_facilities = df[['facilities','filtered_word']]
    df_facilities.rename(columns={'facilities': 'label'}, inplace=True)
    return df_value, df_comfort, df_location, df_cleanliness, df_service, df_facilities

In [180]:
#df_value_train, df_comfort_train, df_location_train, df_cleanliness_train, df_service_train ,df_facilities_train = split_df(df_train)
#df_value_test, df_comfort_test, df_location_test, df_cleanliness_test, df_service_test, df_facilities_test = split_df(df_test)
df_value, df_comfort, df_location, df_cleanliness, df_service ,df_facilities = split_df(df)

In [181]:
df_facilities

,label,filtered_word
0,0,"價格(Na),房間(Nc),老闆娘(Na),人(Na),早餐(Na),旅客(Na),重點(N..."
1,1,"內部(Ncd),房間(Nc),場地(Na),團體(Na)"
2,0,房間(Nc)
3,1,"房子(Na),房間(Nc),採光(Na),大廳(Nc),氣派(Na),房價(Na),台東(N..."
4,0,"空間(Na),大樓(Na),下(Ncd),吐司(Na),咖啡(Na),老闆(Na),速度(Na)"
...,...,...
1267,1,"港式(Na),餐廳(Nc),口味(Na),環境(Na),機車(Na),汽車(Na),停車位(..."
1268,0,"場地(Na),氣派(Na),丁香魚(Na),蠟味(Na),蘿蔔糕(Na),份量(Na),牛肉..."
1269,1,"交通(Na),地下室(Nc),停車場(Nc),菜色(Na),空間(Na),飯店(Nc)"
1270,0,"地點(Na),逢甲(Nb),夜市(Nc),老闆娘(Na),房間(Nc),浴室(Nc)"


### 清理資料(移除詞性標註的文字)

In [182]:
def remove_N_comma(sentence):
    # 把後面(N..)(V..)(F..)拿掉
    sentence = str(sentence)
    pattern = re.compile(r"\([N,V,F,P,A].*?\)") #移除詞性標示
    sentence = re.sub(pattern, '', sentence)
    pattern = re.compile(r",") #將逗號替換為空格
    sentence = re.sub(pattern, ' ', sentence)
    return sentence
pd.options.mode.chained_assignment = None  # 忽略警告

In [183]:
#訓練集
df_facilities['filtered_word'] = df_facilities.apply(lambda x: remove_N_comma(x['filtered_word']),axis=1)

In [184]:
print(df_facilities.shape)
df_facilities

(1272, 2)


,label,filtered_word
0,0,價格 房間 老闆娘 人 早餐 旅客 重點 早餐
1,1,內部 房間 場地 團體
2,0,房間
3,1,房子 房間 採光 大廳 氣派 房價 台東 民宿
4,0,空間 大樓 下 吐司 咖啡 老闆 速度
...,...,...
1267,1,港式 餐廳 口味 環境 機車 汽車 停車位 高鐵 附近
1268,0,場地 氣派 丁香魚 蠟味 蘿蔔糕 份量 牛肉粥
1269,1,交通 地下室 停車場 菜色 空間 飯店
1270,0,地點 逢甲 夜市 老闆娘 房間 浴室


In [185]:
bool = df_facilities['filtered_word'].str.contains('Ne')
filter_data = df_facilities[bool]
filter_data

,label,filtered_word


## 模型架構

### 套件引用

In [186]:
#import package
#轉向量用
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer 
from scipy.sparse import coo_matrix

from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import cross_validate
import pickle #儲存模型用
from sklearn.model_selection import train_test_split
#類別採樣
import imblearn.over_sampling as over_sampling
import imblearn.under_sampling as under_sampling
import imblearn.combine as combine
from imblearn.pipeline import make_pipeline as make_pipeline_imb


#模型
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
#from xgboost.sklearn import XGBClassifier

#模型效能表現
import sklearn.metrics as metrics

### 顯示訓練結果

In [187]:
def classification_report(y_test, pre):
    #混淆矩陣
    confusion = metrics.confusion_matrix(y_test, pre)
    TP = confusion[1,1]
    TN = confusion[0,0]
    FP = confusion[0,1]
    FN = confusion[1,0]
    print("TP:",TP)
    print("TN:",TN)
    print("FP:",FP)
    print("FN:",FN)
    #Accuracy
    accuracy = (TP+TN)/float(TP+TN+FN+FP)
    print("Accuracy：", accuracy)
    #Sensitivity(Recall)
    recall = TP/float(TP+FN)
    print("Recall：", recall)
    #Specificity
    specificity = TN/float(TN+FP)
    print("Specificity：", specificity)
    #Precision
    precision = TP/float(TP+FP)
    print("Precision：", precision)
    #f1-score
    f1_score = ((2*precision*recall)/(precision+recall))
    print("f1_score：", f1_score)
    #AUC
    print("AUC：", metrics.roc_auc_score(y_test, pre))

In [188]:
df_facilities

,label,filtered_word
0,0,價格 房間 老闆娘 人 早餐 旅客 重點 早餐
1,1,內部 房間 場地 團體
2,0,房間
3,1,房子 房間 採光 大廳 氣派 房價 台東 民宿
4,0,空間 大樓 下 吐司 咖啡 老闆 速度
...,...,...
1267,1,港式 餐廳 口味 環境 機車 汽車 停車位 高鐵 附近
1268,0,場地 氣派 丁香魚 蠟味 蘿蔔糕 份量 牛肉粥
1269,1,交通 地下室 停車場 菜色 空間 飯店
1270,0,地點 逢甲 夜市 老闆娘 房間 浴室


In [189]:
bool = df_facilities['filtered_word'].str.contains('cp')
filter_data = df_facilities[bool]
filter_data

,label,filtered_word


### 切分訓練、測試數據

In [190]:
from sklearn.model_selection import train_test_split
def split_label(df,seed):
    X = df.filtered_word.tolist()
    y = df.label
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=1/3,random_state=seed)
    return X_train,X_test,y_train,y_test

In [191]:
X_train_1,X_test_1,y_train_1,y_test_1 = split_label(df_facilities,1)
X_train_2,X_test_2,y_train_2,y_test_2 = split_label(df_facilities,2)
X_train_3,X_test_3,y_train_3,y_test_3 = split_label(df_facilities,3)

In [192]:
s1 = pd.Series(y_train_1)
freq1 = s1.value_counts() 
print(freq1) 
s2 = pd.Series(y_train_2)
freq2 = s2.value_counts() 
print(freq2) 
s3 = pd.Series(y_train_3)
freq3 = s3.value_counts() 
print(freq3) 

0    638
1    210
Name: label, dtype: int64
0    639
1    209
Name: label, dtype: int64
0    652
1    196
Name: label, dtype: int64


### 模型設計

#### (1) baseline

In [193]:
def SVM_model(X_train,X_test,y_train,y_test):
    print("SVM baseline")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(max_df=0.8,min_df=5,dtype=np.float32), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [194]:
def LR_model(X_train,X_test,y_train,y_test):
    print("LR baseline")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(max_df=0.8,min_df=5,dtype=np.float32), LogisticRegression(random_state=0))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [195]:
def RF_model(X_train,X_test,y_train,y_test):
    print("RF baseline")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(max_df=0.8,min_df=5,dtype=np.float32), RandomForestClassifier(max_depth=2, random_state=0))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [196]:
def AdaBoost_model(X_train,X_test,y_train,y_test):
    print("AdaBoost_model")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

#### (2)執行採樣 => 解決類別不平衡 (SVM)

In [217]:
def SVM_model2(X_train,X_test,y_train,y_test):
    print("ADASYN")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.ADASYN(), svm.SVC(kernel='linear'))
    print(model)
    # Get the names of each feature
    model.fit(X_train, y_train)
    feature_names = model.named_steps["tfidfvectorizer"].get_feature_names()
    #pd.DataFrame([transformer.idf_], columns=vectorizer.get_feature_names())
    #print(feature_names)
    #模型預測
    pre = model.predict(X_test)
    df_predict = pd.DataFrame(data={'X_test': X_test, 'y_test': y_test, 'pre':pre}) 
    df_predict.to_csv('../data/0414/predict_facilities.csv', encoding='utf_8_sig', index=False)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [218]:
SVM_model2(X_train_1,X_test_1,y_train_1,y_test_1)

ADASYN
Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()), ('adasyn', ADASYN()),
                ('svc', SVC(kernel='linear'))])
TP: 60
TN: 275
FP: 49
FN: 40
Accuracy： 0.7900943396226415
Recall： 0.6
Specificity： 0.8487654320987654
Precision： 0.5504587155963303
f1_score： 0.5741626794258374
AUC： 0.7243827160493828




In [199]:
def SVM_model3(X_train,X_test,y_train,y_test):
    print("SMOTE")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.SMOTE(), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [200]:
def SVM_model4(X_train,X_test,y_train,y_test):
    print("RandomOverSampler")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.RandomOverSampler(), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [201]:
def SVM_model5(X_train,X_test,y_train,y_test):
    print("RandomUnderSampler")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), under_sampling.RandomUnderSampler(), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

#### (3)執行採樣 => 解決類別不平衡 (Adaboost)

In [202]:
def AdaBoost_model2(X_train,X_test,y_train,y_test):
    print("ADASYN")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.ADASYN(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [203]:
def AdaBoost_model3(X_train,X_test,y_train,y_test):
    print("SMOTE")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.SMOTE(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [204]:
def AdaBoost_model4(X_train,X_test,y_train,y_test):
    print("RandomOverSampler")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.RandomOverSampler(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [205]:
def AdaBoost_model5(X_train,X_test,y_train,y_test):
    print("RandomUnderSampler")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), under_sampling.RandomUnderSampler(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

## 模型訓練&結果

### 資料集1

#### baseline

In [206]:
SVM_model(X_train_1,X_test_1,y_train_1,y_test_1)
LR_model(X_train_1,X_test_1,y_train_1,y_test_1)
RF_model(X_train_1,X_test_1,y_train_1,y_test_1)
AdaBoost_model(X_train_1,X_test_1,y_train_1,y_test_1)

SVM baseline
TP: 29
TN: 316
FP: 8
FN: 71
Accuracy： 0.8136792452830188
Recall： 0.29
Specificity： 0.9753086419753086
Precision： 0.7837837837837838
f1_score： 0.4233576642335766
AUC： 0.6326543209876544


LR baseline
TP: 19
TN: 319
FP: 5
FN: 81
Accuracy： 0.7971698113207547
Recall： 0.19
Specificity： 0.9845679012345679
Precision： 0.7916666666666666
f1_score： 0.3064516129032258
AUC： 0.5872839506172839


RF baseline
TP: 0
TN: 324
FP: 0
FN: 100
Accuracy： 0.7641509433962265
Recall： 0.0
Specificity： 1.0
Precision： nan
f1_score： nan
AUC： 0.5


AdaBoost_model


C:\Users\g3190\AppData\Local\Temp/ipykernel_3316/3777238089.py:22: RuntimeWarning: invalid value encountered in true_divide
  precision = TP/float(TP+FP)


TP: 41
TN: 282
FP: 42
FN: 59
Accuracy： 0.7617924528301887
Recall： 0.41
Specificity： 0.8703703703703703
Precision： 0.4939759036144578
f1_score： 0.44808743169398907
AUC： 0.6401851851851852




#### 處理類別不平衡(SVM)

In [207]:
SVM_model2(X_train_1,X_test_1,y_train_1,y_test_1)
SVM_model3(X_train_1,X_test_1,y_train_1,y_test_1)
SVM_model4(X_train_1,X_test_1,y_train_1,y_test_1)
SVM_model5(X_train_1,X_test_1,y_train_1,y_test_1)

ADASYN
Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()), ('adasyn', ADASYN()),
                ('svc', SVC(kernel='linear'))])
['1樓', '2樓', '5星級', '8點', '9點半', 'cp值', 'nan', '一下', '一品', '一家人', '一樓', '一次性', '丁香魚', '三明治', '三樓', '三溫暖', '三義', '三龜', '上限', '下午茶', '下雨天', '世界', '世紀', '中國風', '中央', '中心', '中心點', '中正', '中班', '中繼站', '中華', '中餐', '中餐廳', '丹麥', '主人', '主人家', '主題', '主題房', '之後', '九份', '乳豬', '事項', '二樓', '二館', '五星級', '交流道', '交誼廳', '交通', '人員', '人工', '人情味', '人房', '人潮', '人間', '今天', '今日', '仙境', '仙草', '份量', '休息區', '休閒', '位子', '位置', '低海拔', '低音', '低頻', '住宅區', '住客', '住家', '住宿', '住宿區', '住宿費', '住所', '佩佩', '來來', '來客', '便利', '便利商店', '便利性', '便當', '保護區', '信息', '個性', '假日', '假期', '停車', '停車位', '停車場', '健身室', '健身房', '備品', '備品量', '傳藝', '價位', '價值', '價廉', '價格', '價錢', '優勢', '優格', '優點', '充電線', '先生', '兒子', '兒童', '兔子', '內外', '內容', '內涵', '內部', '全家', '全聯', '公共區', '公園', '公尺', '公廁', '公益路', '公車', '公車站', '公道', '六合', '六星', '六星級', '六福村', '冬天', '冬瓜', '冰淇淋', '冰箱', '冷氣', '冷氣室', '冷盤', '冷菜', '凱特', '分鐘', '切仔麵', '別墅', '前面', 

#### 處理類別不平衡(Adaboost)

In [208]:
AdaBoost_model2(X_train_1,X_test_1,y_train_1,y_test_1)
AdaBoost_model3(X_train_1,X_test_1,y_train_1,y_test_1)
AdaBoost_model4(X_train_1,X_test_1,y_train_1,y_test_1)
AdaBoost_model5(X_train_1,X_test_1,y_train_1,y_test_1)

ADASYN
TP: 53
TN: 285
FP: 39
FN: 47
Accuracy： 0.7971698113207547
Recall： 0.53
Specificity： 0.8796296296296297
Precision： 0.5760869565217391
f1_score： 0.5520833333333334
AUC： 0.7048148148148149


SMOTE
TP: 57
TN: 283
FP: 41
FN: 43
Accuracy： 0.8018867924528302
Recall： 0.57
Specificity： 0.8734567901234568
Precision： 0.5816326530612245
f1_score： 0.5757575757575758
AUC： 0.7217283950617284


RandomOverSampler
TP: 40
TN: 270
FP: 54
FN: 60
Accuracy： 0.7311320754716981
Recall： 0.4
Specificity： 0.8333333333333334
Precision： 0.425531914893617
f1_score： 0.4123711340206186
AUC： 0.6166666666666667


RandomUnderSampler
TP: 53
TN: 227
FP: 97
FN: 47
Accuracy： 0.660377358490566
Recall： 0.53
Specificity： 0.7006172839506173
Precision： 0.35333333333333333
f1_score： 0.424
AUC： 0.6153086419753087




### 資料集2

#### baseline

In [209]:
SVM_model(X_train_2,X_test_2,y_train_2,y_test_2)
LR_model(X_train_2,X_test_2,y_train_2,y_test_2)
RF_model(X_train_2,X_test_2,y_train_2,y_test_2)
AdaBoost_model(X_train_2,X_test_2,y_train_2,y_test_2)

SVM baseline
TP: 26
TN: 314
FP: 9
FN: 75
Accuracy： 0.8018867924528302
Recall： 0.25742574257425743
Specificity： 0.9721362229102167
Precision： 0.7428571428571429
f1_score： 0.3823529411764706
AUC： 0.6147809827422371


LR baseline
TP: 15
TN: 317
FP: 6
FN: 86
Accuracy： 0.7830188679245284
Recall： 0.1485148514851485
Specificity： 0.9814241486068112
Precision： 0.7142857142857143
f1_score： 0.2459016393442623
AUC： 0.5649695000459798


RF baseline
TP: 0
TN: 323
FP: 0
FN: 101
Accuracy： 0.7617924528301887
Recall： 0.0
Specificity： 1.0
Precision： nan
f1_score： nan
AUC： 0.5


AdaBoost_model


C:\Users\g3190\AppData\Local\Temp/ipykernel_3316/3777238089.py:22: RuntimeWarning: invalid value encountered in true_divide
  precision = TP/float(TP+FP)


TP: 38
TN: 299
FP: 24
FN: 63
Accuracy： 0.7948113207547169
Recall： 0.37623762376237624
Specificity： 0.9256965944272446
Precision： 0.6129032258064516
f1_score： 0.4662576687116564
AUC： 0.6509671090948104




#### 處理類別不平衡(SVM)

In [210]:
SVM_model2(X_train_2,X_test_2,y_train_2,y_test_2)
SVM_model3(X_train_2,X_test_2,y_train_2,y_test_2)
SVM_model4(X_train_2,X_test_2,y_train_2,y_test_2)
SVM_model5(X_train_2,X_test_2,y_train_2,y_test_2)

ADASYN
Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()), ('adasyn', ADASYN()),
                ('svc', SVC(kernel='linear'))])
['1樓', '2樓', '30', '5星級', '8點', 'cp值', 'nan', '一下', '一中', '一品', '一家人', '一樓', '丁香魚', '三明治', '三樓', '三溫暖', '三義', '上方', '上限', '下午茶', '下雨天', '世界', '世紀', '中下', '中國風', '中央', '中山', '中心', '中心點', '中秋', '中華', '中間', '中餐', '丸子', '丹麥', '主人', '主廚', '主題', '主題房', '之上', '九份', '九宮格', '乳豬', '事項', '二樓', '二館', '五星級', '交流道', '交誼廳', '交通', '人員', '人情味', '人房', '人潮', '人間', '今天', '今日', '今早', '仙境', '仙草', '以上', '以外', '份量', '企業', '休息區', '伯朗大道', '位子', '位置', '住宅區', '住家', '住宿', '住宿區', '住宿費', '住房', '作品', '作風', '佩佩', '來來', '來客', '便利', '便利商店', '便當', '保護區', '保險箱', '信息', '個性', '假日', '假期', '停車', '停車位', '停車場', '健身室', '健身房', '備品', '備品量', '傢俱', '傳藝', '價位', '價值', '價廉', '價格', '價比', '價錢', '優勢', '優格', '優點', '充電線', '先生', '光復鄉', '免治', '兒童', '內側', '內容', '內涵', '內部', '全家', '全聯', '公共區', '公園', '公廁', '公東', '公益路', '公車站', '公道', '六合', '六星', '六星級', '六樓', '六福村', '冬天', '冰水', '冰淇淋', '冰箱', '冷卻', '冷暖器', '冷气', '冷氣', '冷氣

#### 處理類別不平衡(Adaboost)

In [211]:
AdaBoost_model2(X_train_2,X_test_2,y_train_2,y_test_2)
AdaBoost_model3(X_train_2,X_test_2,y_train_2,y_test_2)
AdaBoost_model4(X_train_2,X_test_2,y_train_2,y_test_2)
AdaBoost_model5(X_train_2,X_test_2,y_train_2,y_test_2)

ADASYN
TP: 46
TN: 296
FP: 27
FN: 55
Accuracy： 0.8066037735849056
Recall： 0.45544554455445546
Specificity： 0.9164086687306502
Precision： 0.6301369863013698
f1_score： 0.528735632183908
AUC： 0.6859271066425529


SMOTE
TP: 40
TN: 291
FP: 32
FN: 61
Accuracy： 0.7806603773584906
Recall： 0.39603960396039606
Specificity： 0.9009287925696594
Precision： 0.5555555555555556
f1_score： 0.4624277456647399
AUC： 0.6484841982650277


RandomOverSampler
TP: 38
TN: 289
FP: 34
FN: 63
Accuracy： 0.7712264150943396
Recall： 0.37623762376237624
Specificity： 0.8947368421052632
Precision： 0.5277777777777778
f1_score： 0.4393063583815029
AUC： 0.6354872329338197


RandomUnderSampler
TP: 49
TN: 243
FP: 80
FN: 52
Accuracy： 0.6886792452830188
Recall： 0.48514851485148514
Specificity： 0.7523219814241486
Precision： 0.3798449612403101
f1_score： 0.4260869565217391
AUC： 0.6187352481378169




### 資料集3

#### baseline

In [212]:
SVM_model(X_train_3,X_test_3,y_train_3,y_test_3)
LR_model(X_train_3,X_test_3,y_train_3,y_test_3)
RF_model(X_train_3,X_test_3,y_train_3,y_test_3)
AdaBoost_model(X_train_3,X_test_3,y_train_3,y_test_3)

SVM baseline
TP: 30
TN: 307
FP: 3
FN: 84
Accuracy： 0.7948113207547169
Recall： 0.2631578947368421
Specificity： 0.9903225806451613
Precision： 0.9090909090909091
f1_score： 0.4081632653061224
AUC： 0.6267402376910017


LR baseline
TP: 15
TN: 309
FP: 1
FN: 99
Accuracy： 0.7641509433962265
Recall： 0.13157894736842105
Specificity： 0.9967741935483871
Precision： 0.9375
f1_score： 0.23076923076923075
AUC： 0.5641765704584041


RF baseline
TP: 0
TN: 310
FP: 0
FN: 114
Accuracy： 0.7311320754716981
Recall： 0.0
Specificity： 1.0
Precision： nan
f1_score： nan
AUC： 0.5


AdaBoost_model


C:\Users\g3190\AppData\Local\Temp/ipykernel_3316/3777238089.py:22: RuntimeWarning: invalid value encountered in true_divide
  precision = TP/float(TP+FP)


TP: 51
TN: 264
FP: 46
FN: 63
Accuracy： 0.7429245283018868
Recall： 0.4473684210526316
Specificity： 0.8516129032258064
Precision： 0.5257731958762887
f1_score： 0.48341232227488157
AUC： 0.649490662139219




#### 處理類別不平衡(SVM)

In [213]:
SVM_model2(X_train_3,X_test_3,y_train_3,y_test_3)
SVM_model3(X_train_3,X_test_3,y_train_3,y_test_3)
SVM_model4(X_train_3,X_test_3,y_train_3,y_test_3)
SVM_model5(X_train_3,X_test_3,y_train_3,y_test_3)

ADASYN
Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()), ('adasyn', ADASYN()),
                ('svc', SVC(kernel='linear'))])
['1樓', '2樓', '30', '3樓', '5星級', '5點半', '8點', '9點半', 'cp值', 'nan', '一下', '一中', '一家人', '一樓', '一陣子', '丁香魚', '三明治', '三樓', '三溫暖', '三龜', '上方', '上限', '上面', '下午茶', '下雨天', '下雨車', '世界', '世紀', '中下', '中國風', '中心', '中心點', '中正', '中班', '中秋', '中繼站', '中間', '中餐', '中餐廳', '丸子', '丹麥', '主人', '主人家', '主廚', '主題', '主題房', '之上', '之後', '九份', '九宮格', '乳豬', '二三樓', '二樓', '二館', '五星級', '交流道', '交誼廳', '交通', '人員', '人工', '人情味', '人房', '人數', '人文', '人潮', '人間', '今天', '今日', '今早', '仙境', '仙草', '以下', '以外', '份量', '企業', '休息區', '休閒', '位子', '位置', '低海拔', '低音', '低頻', '住宅區', '住客', '住家', '住宿', '住宿費', '住所', '作風', '來客', '便利', '便利商店', '便利性', '便當', '保護區', '保險箱', '信息', '個資', '假日', '假期', '停車', '停車位', '停車場', '健身室', '健身房', '備品', '傢俱', '價位', '價值', '價廉', '價格', '價比', '價錢', '優勢', '優格', '優點', '充電線', '先生', '免治', '兒子', '兒童', '兔子', '內外', '內容', '內涵', '內部', '全家', '全聯', '公共區', '公園', '公廁', '公益路', '公車站', '六合', '六星', '六星級', '六樓', '

#### 處理類別不平衡(Adaboost)

In [214]:
AdaBoost_model2(X_train_3,X_test_3,y_train_3,y_test_3)
AdaBoost_model3(X_train_3,X_test_3,y_train_3,y_test_3)
AdaBoost_model4(X_train_3,X_test_3,y_train_3,y_test_3)
AdaBoost_model5(X_train_3,X_test_3,y_train_3,y_test_3)

ADASYN
TP: 56
TN: 277
FP: 33
FN: 58
Accuracy： 0.785377358490566
Recall： 0.49122807017543857
Specificity： 0.8935483870967742
Precision： 0.6292134831460674
f1_score： 0.5517241379310345
AUC： 0.6923882286361064


SMOTE
TP: 59
TN: 277
FP: 33
FN: 55
Accuracy： 0.7924528301886793
Recall： 0.5175438596491229
Specificity： 0.8935483870967742
Precision： 0.6413043478260869
f1_score： 0.5728155339805826
AUC： 0.7055461233729485


RandomOverSampler
TP: 59
TN: 275
FP: 35
FN: 55
Accuracy： 0.7877358490566038
Recall： 0.5175438596491229
Specificity： 0.8870967741935484
Precision： 0.6276595744680851
f1_score： 0.5673076923076923
AUC： 0.7023203169213356


RandomUnderSampler
TP: 77
TN: 172
FP: 138
FN: 37
Accuracy： 0.5872641509433962
Recall： 0.6754385964912281
Specificity： 0.5548387096774193
Precision： 0.3581395348837209
f1_score： 0.4680851063829787
AUC： 0.6151386530843237


